In [1]:
%matplotlib inline

import datetime
import os
from glob import glob

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import matplotlib.patheffects as pe
from mpl_toolkits.axes_grid1 import make_axes_locatable
import numpy as np
import pandas as pd
import tqdm.notebook as tqdm
import warnings
from scipy import optimize

from obspy import UTCDateTime, read
from obspy.clients.fdsn import Client
from obspy.clients.fdsn.client import FDSNNoDataException
from obspy.signal import PPSD

%load_ext autoreload
%autoreload 2
from seismic_noise import *
import seismosocialdistancing

# Paramètre initiaux

In [2]:
PPSD_FOLDER  = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/PPSD"
DRMS_FOLDER  = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/DRMS"
MSEED_FOLDER = "C:/Users/Arnaud/Desktop/Travail/Projet de Recherche/SeismicNoiseData/MSEED"
DATA_FOLDER  = "DATA"
#freqs = [(0.01, 0.03), (0.1, 0.25), (0.3, 1), (1, 3), (5, 15), (20, 50)] # Bandes de fréquences à étudier
freq = "6.0-8.0"

tStart = "2013-06-01"
tEnd   = "2015-06-01"

list_stations = ["G.DRV.00.BHZ_6_8Hz"]
#list_stations = np.loadtxt(DATA_FOLDER + "/st_metadata/stations_antarctic.txt", dtype=str)

In [3]:
events = {
    "Winter 1" :      "2019-03-01",
    "Summer 1" :      "2019-10-01",
    "Winter 2" :      "2020-03-01",
    "Summer 2" :      "2020-10-01"
}

# Médiane à toutes les stations par heure

In [4]:
tStart = pd.Timestamp(tStart)
tEnd = pd.Timestamp(tEnd)
nDays = (tEnd - tStart).days
dates = pd.date_range(tStart, periods=nDays, freq="D")
dfDates = pd.DataFrame(index=dates)

In [5]:
#décalage horaire été CCD
#deca_hor=7

#décalage horaire été DRV
deca_hor=10

#indice pour concatener la matrice pour prendre en compte le décalage horaire
indice=deca_hor

h_list= [ "0:00", "1:00", "2:00", "3:00", "4:00", "5:00", "6:00", "7:00", "8:00", "9:00", "10:00",
           "11:00", "12:00", "13:00", "14:00", "15:00", "16:00", "17:00", "18:00", "19:00", "20:00",
           "21:00", "22:00", "23:00"]
h_mid1= np.arange(0, 24, 1)
h_mid = []
h_mid = np.append(h_mid1[indice:],h_mid1[:indice])


for k in tqdm.tqdm(range(len(h_list))):

    med_values = []

    for j, st_str in enumerate(list_stations):
        # name = "{}_{}_{}".format(st_str, str(periods[i][0])[:10], str(periods[i][1])[:10])

        ### Calcul du déplacement RMS de la station
        try:
            dataframe = pd.read_csv("{}/{}.csv".format(DRMS_FOLDER, st_str), index_col=0)
            dataframe.index = pd.to_datetime(dataframe.index)
            dRMS = dataframe
        except:
            continue

        ### On garde les données dans la période temporelle qui nous intéresse
        mask_time = (dataframe.index > tStart) & (dataframe.index <= tEnd)
        dRMS = dRMS.loc[mask_time]

        ### Récupération du bruit
        dRMS_med = dRMS[freq].between_time(h_list[k-1], h_list[k]).resample("1D").median()
        dRMS_med = dfDates.join(dRMS_med, how='outer')

        ### Normalisation du bruit
        dRMS_med = dRMS_med[freq].tolist()
        norm = np.nanmedian(dRMS_med)
        dRMS_med = dRMS_med/norm
        if len(dRMS_med) > len(dates):
            dRMS_med = dRMS_med[:len(dates)]
            
        # print(len(dRMS_med))
        
        med_values.append(dRMS_med)

    noise_ant = np.zeros(nDays)

    for i in range(nDays):

        sublist = list()

        for j in range(np.shape(med_values)[0]):
            sublist.append(med_values[j][i])

        noise_ant[i] = np.nanmedian(sublist)

    #norm_period = ["2020-01-01", "2020-03-01"]
    #t1_i = np.where(dates >= pd.Timestamp(norm_period[0]))[0][0]
    #t2_i = np.where(dates >= pd.Timestamp(norm_period[1]))[0][0]
    #norm = (np.nanmedian(noise_ant[t1_i:t2_i])-1)*100  # Normaliser à 0 sur la période Mi-Janvier > Mi-Mars
    #noise_ant = (noise_ant-1)*100+norm
    noise_ant = (noise_ant - np.nanmedian(noise_ant))*100

    color = "#353535"
    fig, ax = plt.subplots(figsize=(12, 5))
    ax.tick_params(direction="in", which="both", top=True, right=True)
    # ax.set_title("Frequency band : {}Hz".format(freq))
    
    # décalage de la liste pour ajuster à l'heure locale
    h_list1= []
    h_list1= np.append(h_list[indice:],h_list[:indice])
    
    ax.plot(dates, noise_ant, color=color, lw=2, zorder=3, label="Noise variations at {}h ({}-{})".format(h_mid[k], h_list1[k-1], h_list1[k]))
    # plt.ylim(1.7*np.nanquantile(noise_fr, 0.01), 1.5*np.nanquantile(noise_fr, 0.99))
    # plt.ylim(3.5 * np.quantile(noise_fr, 0.05), 5 * np.quantile(noise_fr, 0.95))
    plt.ylim(-200, 700)
    ax.set_xlim([dates[0], dates[-1]])
    # ax.set_xlabel("Year 2020")
    ax.set_ylabel("Noise level variations [%]")

    '''     for desc, t_search in events.items():
        t_search = pd.Timestamp(t_search)
        t_text = t_search - pd.Timedelta("5 days")
        ax.axvline(t_search, ls="--", lw=1, color="#B82A2A", zorder=10)   # Confinement 1
        # ax.text(t_text, 2.8 * np.quantile(noise_fr, 0.05), desc, color="#396790", fontsize="medium", rotation=90)   # Confinement 1

    med_var = med_values

    for i in range(np.shape(med_var)[0]):
        sublist = med_var[i][:]
        # med_var[i][:] = (sublist - np.nanmedian(sublist[t1_i:t2_i]))*100
        med_var[i][:] = (sublist - np.nanmedian(sublist))*100

    std = np.nanstd(med_var, axis=0)
    plt.fill_between(dates,noise_fr-std,noise_fr+std,alpha=.25)'''
        
    plt.grid(color="#D5D5D5", linestyle='--', linewidth=1)
    plt.legend(loc=2)
    #plt.title("std = {}".format(round(np.nanmedian(std),3)))
    plt.tight_layout()
    plt.savefig("RESULTS/ant_hour/DRV/2013-2015/noise_{}Hz_{}h_2013_2015.png".format(freq, h_mid[k]), dpi=300)
    plt.clf()
    plt.close()

100%|██████████| 24/24 [00:15<00:00,  1.55it/s]
